# Lab 3: Stochastic Gradient Descent

The goal of this lab session is to code an optimization algorithm that optimzes the penalized loss function of the logistic regression model.

You have to send the filled notebook named **"L3_familyname1_familyname2.ipynb"** (groups of 2) by email to aml.centralesupelec.2019@gmail.com by October 17, 2019. Please put **"AML-L3"** in the subject. 

We begin with the standard imports:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

We import the dataset that we are going to use, an indian dataset including in the last column information about the diabetes status of patients:

In [2]:
from sklearn import model_selection

diabetes_data = pd.read_csv("diabetes_data.csv", sep=",")

diabetes_train, diabetes_test = model_selection.train_test_split(diabetes_data)
diabetes_train_x = diabetes_train.iloc[:, :-1].values
diabetes_train_y = diabetes_train.iloc[:, -1].values
diabetes_train_y[diabetes_train_y == 0] = -1

diabetes_test_x = diabetes_test.iloc[:, :-1].values
diabetes_test_y = diabetes_test.iloc[:, -1].values
diabetes_test_y[diabetes_test_y == 0] = -1

## Logistic Regression



Today we’ll be moving from linear regression to logistic regression, one of the simplest ways to deal with a classification problem. Instead of fitting a line, logistic regression models the probability that the outcome is 1 given the value of the predictor. In order to do this we need a function that transforms our predictor variable to a value between 0 and 1. Lots of functions can do that, but the logistic function is the most common choice:

$$f(z) = \frac{1}{1+\exp{-z}}.$$

To predict the class of our observations we'll have to minimize the corresponding loss function and as we are in a high-dimensional context we'll add an $l_2$ regularization to the model:

$$L(\textbf{w}) = \sum_{i=1}^n log(1+\exp(-y_i\textbf{w}^Tx_i))+\frac{\lambda}{2} \| \textbf{w} \|^2,$$

where $x_i$ is the vector of features for the observation $i$ and $y_i \in \{-1, 1\}$ is the class label.  


We first use the `sklearn` implementation:

In [3]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty="l2", C=2) 
model.fit(diabetes_train_x, diabetes_train_y)
y_pred = model.predict(diabetes_test_x)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


and we compute the accuracy score to evaluate the model performance:

In [4]:
from sklearn.metrics import accuracy_score
accuracy_score(diabetes_test_y, y_pred)

0.7604166666666666

### Assignment

Implement from scratch your own logistic regression model with stochastic gradient descent optimization. 

- Fill in the class

- Display the evolution of the cost function along iterations. Do this for several strategies for the setting of the learning rate

- Try the different acceleration strategies

- Train the model with the training set and evaluate its performance in the test set

In [46]:
class StochasticLogisticRegression():
    """ Class for logistic regression:
    
    Attributes:
    -----------
    coef_: 1-dimensional np.array
        coefficients 
    alpha_: float
        regularization parameter
    lr_: float
        the learning rate
    bsize: integer
        the size of the mini-batch >=1
    coef_history_: list
        the list of all visited betas
    f_history_: list 
        the list of all evaluations in visited betas
    """
    def __init__(self, alpha, lr=1e-1, batch_size=50, max_iter=500):
        self.coef_  = None
        self.alpha_ = alpha
        self.lr_ = lr
        self.batch_size_ = batch_size
        self.max_iter_ = max_iter
        self.coef_history_ = []
        self.f_history_ = []

    def logistic(self, z):
        # logistic function
        return 1/(1+np.exp(-z))
    
    def set_lr(self, itr, C=2):
        self.lr_ = C / itr
    
        
    def fit(self, X, y, start):
        """ Fit the data (X, y).
    
        Parameters:
        -----------
        X: (num_samples, num_features) np.array
            Design matrix
        y: (num_sampes, ) np.array
            Output vector
        
        Note:
        -----
        Updates self.coef_
        """
        
        def f_lr(beta):
            '''evaluate the F=\sum_{i=1}^n f_i in beta'''
            F = 0
            for i in range(X.shape[0]):
                F += np.log(1 + np.exp(-y[i]*np.matmul(np.transpose(X[i]), beta))) + self.alpha_*0.5*np.matmul(np.transpose(beta), beta)
            
            return F
        
        def grad_logistic_loss(beta, batch):
            grad = 0
            for sample_idx in batch:
                y_i = y[sample_idx]
                x_i = X[sample_idx]
                grad += - y_i * x_i / (1 + np.exp(y_i*np.matmul(np.transpose(x_i), beta)))
            return grad / len(batch) + self.alpha_*beta
        
        
        self.coef_ = start
        self.coef_history_.append(self.coef_)
        self.f_history_.append(f_lr(self.coef_))
        
        # Batches creation
        batches = []
        indexes = list(range(X.shape[0]))
        while len(indexes) > 0:
            batch = []
            while len(batch) < self.batch_size_ and len(indexes) > 0:
                i = np.random.randint(low=0,high=len(indexes),size=1)[0]
                batch.append(indexes.pop(i))
            batches.append(batch)
            
            
        itr = 0
        while itr < self.max_iter_: # critere de convergence
            itr += 1
            self.set_lr(itr)
            self.coef_ = self.coef_ - self.lr_ * grad_logistic_loss(self.coef_, batch=batches[itr%len(batches)])

            # Update the memory
            self.coef_history_.append(self.coef_)
            self.f_history_.append(f_lr(self.coef_))
            
         
    def predict(self, X):
        """ Make binary predictions for data X.
    
        Parameters:
        -----------
        X: (num_samples, num_features) np.array
            Design matrix
        
        Returns:
        -----
        y_pred: (num_samples, ) np.array
            Predictions (0 or 1)
        """
        y_pred = self.logistic(np.matmul(X,self.coef_)) > 0.5
        y_pred = y_pred.astype(int)
        
        return y_pred

Apply to the data

In [57]:
my_model = StochasticLogisticRegression(alpha=.05, lr=1e-3, batch_size=50, max_iter=5000) 
my_model.fit(diabetes_train_x, diabetes_train_y, start=np.random.rand(diabetes_train_x.shape[1]))
y_pred = my_model.predict(diabetes_test_x)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in exp
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: overflow encountered in exp


and we compute the accuracy score to evaluate the model performance:

In [58]:
from sklearn.metrics import accuracy_score
accuracy_score(diabetes_test_y, y_pred)

0.3385416666666667

Comment the results

Implement only one acceleration method and compare the results